<a href="https://colab.research.google.com/github/pierretfie/python_world/blob/main/image%20detection/deepface_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/pierretfie/python_world.git
%cd /content/python_world/image detection/
# !pip install -r requirements.txt

Cloning into 'python_world'...
remote: Enumerating objects: 1236, done.
remote: Counting objects: 100% (1003/1003), done.
remote: Compressing objects: 100% (784/784), done.
remote: Total 1236 (delta 218), reused 999 (delta 215), pack-reused 233 (from 1)
Receiving objects: 100% (1236/1236), 49.45 MiB | 17.57 MiB/s, done.
Resolving deltas: 100% (322/322), done.
/content/python_world/image detection


In [2]:
!pip install --upgrade h5py
!pip install deepface tensorflow==2.9.1 keras==2.9.0



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 56.4 MB/s eta 0:00:00
  Attempting uninstall: h5py
    Found existing installation: h5py 3.11.0
    Uninstalling h5py-3.11.0:
      Successfully uninstalled h5py-3.11.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.7/511.7 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 75.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
#mount google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
from deepface import DeepFace
import os
import pickle

# Dictionary to store embeddings
embeddings_db = {}
reference_folder = '/content/drive/MyDrive/reference_images'

# Loop through reference images
for person in os.listdir(reference_folder):
    person_folder = os.path.join(reference_folder, person)
    embeddings_db[person] = []

    for img_file in os.listdir(person_folder):
        img_path = os.path.join(person_folder, img_file)

        # Generate embeddings and store only the embedding array
        result = DeepFace.represent(img_path=img_path, model_name="VGG-Face", enforce_detection=False)
        embedding = result[0]["embedding"]  # Extract only the embedding array
        embeddings_db[person].append(embedding)

# Save embeddings_db to a file
with open("/content/drive/MyDrive/embeddings_db.pkl", "wb") as f:
    pickle.dump(embeddings_db, f)


In [14]:
import numpy as np
from deepface import DeepFace

# Function for identifying a single person
def ID_single_person(img_path, embeddings_db, threshold=0.6):
    # Generate embedding for the input image
    result = DeepFace.represent(img_path=img_path, model_name='VGG-Face', enforce_detection=False)
    embedding = np.array(result[0]["embedding"])  # Ensure embedding is a numeric array

    # Compare embedding to embeddings in the database
    for person, embeddings in embeddings_db.items():
        for ref_embedding in embeddings:
            ref_embedding = np.array(ref_embedding)  # Ensure ref_embedding is also a numeric array
            distance = np.linalg.norm(ref_embedding - embedding)
            if distance < threshold:
                return person  # Match found

    return "Unknown, no match found"


# Example usage
new_image_path = '/content/drive/MyDrive/reference_images/Peter/DSC_0299~2.JPG'
identified_person = ID_single_person(new_image_path, embeddings_db)
print(f'Person identified as {identified_person}')


Person identified as Peter
